# Data processing

In [1]:
! rsync -a /kaggle/input/mmdetection-v280/mmdetection /
! pip install /kaggle/input/mmdetection-v280/src/mmpycocotools-12.0.3/mmpycocotools-12.0.3/
! pip install /kaggle/input/hpapytorchzoo/pytorch_zoo-master/
! pip install /kaggle/input/hpacellsegmentation/HPA-Cell-Segmentation/
! pip install /kaggle/input/iterative-stratification/iterative-stratification-master/

! cp -r /kaggle/input/kgl-humanprotein-data/kgl_humanprotein_data /
! cp -r /kaggle/input/humanpro/kgl_humanprotein /

%cd /kgl_humanprotein/nbs/
import sys
sys.path.append('/kgl_humanprotein/')

Processing /kaggle/input/mmdetection-v280/src/mmpycocotools-12.0.3/mmpycocotools-12.0.3
  Created wheel for mmpycocotools: filename=mmpycocotools-12.0.3-cp37-cp37m-linux_x86_64.whl size=272913 sha256=c75ee35b5f2406e8d89c47b327e66cb4cc2c91bd4d2d9b4577d05560e8ff941d
  Stored in directory: /root/.cache/pip/wheels/80/e0/da/3288fdf3965b5c9090f368462db9d28be2c82013f51821090a
Successfully built mmpycocotools
Processing /kaggle/input/hpapytorchzoo/pytorch_zoo-master
  Created wheel for pytorch-zoo: filename=pytorch_zoo-0.0.0-py3-none-any.whl size=30139 sha256=38290172fe3b0bccfc0a1d41d1b2951efa979169ca7057ac9e05f56e5ae7a60a
  Stored in directory: /root/.cache/pip/wheels/7f/18/21/aff5a8914e22461b2b025a9629c2b70464c36183caaf12bc09
Successfully built pytorch-zoo
Processing /kaggle/input/hpacellsegmentation/HPA-Cell-Segmentation
  Created wheel for hpacellseg: filename=hpacellseg-0.1.8-py3-none-any.whl size=14815 sha256=b39f7dbe829cade9d90f0df697b8de9735932a1fb3a893160fd11929058f6504
  Stored in di

In [2]:
from pathlib import Path
import shutil
import zipfile
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from kgl_humanprotein.utils.common_util import *
from kgl_humanprotein.config.config import *
from kgl_humanprotein.data_process import *

run on 61bf9d1683e7


In [3]:
# import holoviews as hv
# hv.extension('bokeh')

In [4]:
dir_hpa = Path('/kaggle/input/hpa-single-cell-image-classification')
dir_trn = dir_hpa/'train'
dir_test = dir_hpa/'test'

## Segment `train` images

In [5]:
# dir_seg_trn = Path('/kaggle/input/hpa-processed-train-dataframe-with-cellwise-rle')
# pth_seg_trn = dir_seg_trn / 'train_df_w_masks_shape_bboxes.csv'

In [6]:
# df_seg_trn = load_seg_trn(pth_seg_trn)

In [7]:
# # This takes ~ 3 hours
# df_cells_trn = split_cells(df_seg_trn)

In [8]:
# df_cells_trn.to_feather('/kaggle/working/df_cells_trn.feather')

In [9]:
print('Loading cell segmentations...')

df_cells_trn = pd.read_feather('/kaggle/input/humanpro-df-cells-trn/df_cells_trn.feather')

Loading cell segmentations...


## Generate `train` crops

10 cells at image size 384 equals 1.9 MB.

In [10]:
crop_sz = 384

In [11]:
subset_sz = 100_000

In [12]:
idx_subset = 4

df_cells_trn_subset = df_cells_trn.iloc[idx_subset * subset_sz: (idx_subset + 1) * subset_sz]
df_cells_trn_subset.reset_index(inplace=True, drop=True)

df_cells_trn_subset.head()

,Id,rle,bbox,Target
0,d17f83aa-bb99-11e8-b2b9-ac1f6b6435d0_10,{'counts': b'iff8>bo13M1N3M2N2O1N2N2N8H4L2N2N:...,"[139, 486, 786, 930]",2|0
1,d17f83aa-bb99-11e8-b2b9-ac1f6b6435d0_11,{'counts': b'bfVc22mo11J600[LNeTNg0k0UOaN7ok1b...,"[1331, 546, 1512, 750]",2|0
2,d17f83aa-bb99-11e8-b2b9-ac1f6b6435d0_12,{'counts': b'`c`d11oo13L9G2N2N4Jb1YN9O1O3M>A2O...,"[840, 555, 1298, 1110]",2|0
3,d17f83aa-bb99-11e8-b2b9-ac1f6b6435d0_13,{'counts': b'ceVk1V1gn16L2N2Gc0C8I3L3L4M6K3L3K...,"[947, 571, 1173, 817]",2|0
4,d17f83aa-bb99-11e8-b2b9-ac1f6b6435d0_14,{'counts': b'nddW2h1Xn10O2N2H=F4O2N2N4L2N1O2N2...,"[1146, 581, 1339, 794]",2|0


In [13]:
%%time

print('Writing out crops...')

dir_crops_trn = DATA_DIR / 'train' / f'images_{crop_sz}'
dir_crops_trn.mkdir(exist_ok=True, parents=True)

df_cells_trn_subset = generate_crops(
    df_cells_trn_subset, dir_trn, dir_crops_trn, out_sz=crop_sz)

Writing out crops...
CPU times: user 51min 33s, sys: 6min 1s, total: 57min 34s
Wall time: 53min 28s


In [14]:
print('Writing out df_cells_trn_subset...')

dir_data_raw = DATA_DIR / 'raw'
dir_data_raw.mkdir(exist_ok=True)

df_cells_trn_subset.to_feather(dir_data_raw / 'train.feather')

Writing out df_cells_trn_subset...


## Backup data

In [15]:
print('Writing to Kaggle output...')

dir_kaggle_output = Path('/kaggle/working')

with zipfile.ZipFile(dir_kaggle_output/f'protein_{idx_subset}.zip', mode='w') as f:
    
    for n in dir_data_raw.ls():
        f.write(n)
        
    for n in dir_crops_trn.ls():
        f.write(n)

Writing to Kaggle output...
